In [37]:
import os
import pandas as pd

In [38]:
csv_file_path = '/home/nogaschw/Codeworkout/cleaned_code.csv'
cwd = os.getcwd()

df = pd.read_csv(csv_file_path, sep=',')

In [39]:
cleaned_file_path = '/home/nogaschw/Codeworkout/cleaned_questions.csv'

questions = pd.read_csv(cleaned_file_path, sep=',')
questions = questions[questions.columns[1:]]
questions = questions[questions['max_score'] != 0]

In [42]:
a = df.merge(questions, how='inner', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [4]:
df = df.sort_values('timestamp', ascending=True)
df = df.groupby(['student_id', 'problem_id', 'course_id']).apply(lambda x: pd.Series({
    'ServerTimestamp': x['timestamp'].tolist(),
    'score': x['score'].tolist(),
    'source_code': x['clean_code'].tolist()
})).reset_index()

/tmp/ipykernel_2600147/2430072954.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['student_id', 'problem_id', 'course_id']).apply(lambda x: pd.Series({


In [5]:
def trim_after_max(score, snapshots, ServerTimestamp):
    max_index = score.index(max(score))
    return score[:max_index+1], snapshots[:max_index+1], ServerTimestamp[:max_index+1]

# Apply the function to each row
df['score'], df['source_code'], df['ServerTimestamp'] = zip(*df.apply(lambda row: trim_after_max(row['score'], row['source_code'], row['ServerTimestamp']), axis=1))
df['start_time'] = df['ServerTimestamp'].apply(lambda x: 0 if x[-1] == 100 else 1)
df['struggling'] = df['score'].apply(lambda x: 0 if x[-1] == 100 else 1)
df['max_score'] = [max(i) for i in df['score']]
df = df[df['max_score'] != -1] # remove all the rows contain only locall code (not really the grade)

In [6]:
len(df)

142718

In [7]:
df_basic = df.merge(questions, how='inner', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [8]:
df_basic = df_basic[['student_id', 'problem_id', 'course_id', 'start_time', 'score', 'source_code', 'prompt', 'struggling']]

In [9]:
df_sorted = df_basic.sort_values(by=['student_id', 'start_time'])
df_sorted['row_number'] = df_sorted.groupby('student_id').cumcount() + 1

In [10]:
prev_code = df_sorted['source_code'].tolist()
prev_code.insert(0, [])
prev_q = df_sorted['prompt'].tolist()
prev_q.insert(0, [])
prev_num_q = df_sorted['problem_id'].tolist()
prev_num_q.insert(0, [])
prev_score = df_sorted['score'].tolist()
prev_score.insert(0, [])

In [11]:
df_sorted['prev_code'] = prev_code[:-1] # last question of the last student
df_sorted['prev_question'] = prev_q[:-1]
df_sorted['prev_question_num'] = prev_num_q[:-1]
df_sorted['score'] = prev_score[:-1]

df_sorted = df_sorted[df_sorted['row_number'] != 1]

In [12]:
prev_code = []
prev_question = []
prev_q_num = []
prev_score = []
for i, row in df_sorted.iterrows():
    if row['row_number'] == 2:
        prev_code.append([row['prev_code']])
        prev_question.append([row['prev_question']])
        prev_q_num.append([row['prev_question_num']])
        prev_score.append([row['score']])
    else:
        prev_code.append(prev_code[-1] + [row['prev_code']])
        prev_question.append(prev_question[-1] + [row['prev_question']])
        prev_q_num.append(prev_q_num[-1] + [row['prev_question_num']])
        prev_score.append(prev_score[-1] + [row['score']])

In [13]:
df_sorted['prev_code'] = prev_code
df_sorted['prev_question'] = prev_question
df_sorted['prev_question_num'] = prev_q_num
df_sorted['score'] = prev_score

# Taxonomy

In [14]:
q_with_compu = questions.copy()
q_with_compu['IfElse'] = questions['conditional']
q_with_compu['Loops'] = questions[["loop_counting", "loop_until", "loop_elements", "loop_nested"]].max(axis=1)
q_with_compu['MathOperations'] = questions[["stat_calculate", "assignment"]].max(axis=1)
q_with_compu['LogicOperators'] = 0
q_with_compu['StringOperations'] = questions[["input_str", "input_cast", "output"]].max(axis=1)
q_with_compu["List"]= questions[["list", "list_2d"]].max(axis=1)
q_with_compu['FileOperations'] = questions[["file_read", "file_write"]].max(axis=1)
q_with_compu["Functions"] = questions[["function_call", "function_def", "function_return"]].max(axis=1)
q_with_compu['Dictionary'] = questions["dictionary"]
q_with_compu['Tuple'] = questions[["tuple", "item_set"]].max(axis=1)

q_with_compu = q_with_compu.drop(columns=['input_str', 'input_cast', 'output', 'assignment', 'conditional',
       'function_call', 'function_def', 'function_return', 'loop_counting',
       'loop_until', 'loop_elements', 'loop_nested', 'stat_calculate',
       'file_read', 'file_write', 'list', 'list_2d', 'dictionary', 'item_set',
       'tuple'])


q_with_compu.fillna(0, inplace=True)
questions.fillna(0, inplace=True)

In [15]:
q_with_compu = q_with_compu.rename(columns={'prompt': 'question'})

In [16]:
df_taxonomy = df_sorted.merge(q_with_compu, how='inner', left_on=['problem_id', 'course_id'], right_on=['id', 'course_id'])

In [17]:
df_taxonomy =df_taxonomy[['student_id', 'course_id', 'problem_id', 'prev_code', 'prev_question_num',
                          'prev_question', 'score', 'question', 'struggling', 'type', 'IfElse', 'Loops',
       'MathOperations', 'LogicOperators', 'StringOperations', 'List', 'FileOperations', 'Functions', 'Dictionary', 'Tuple']]

In [36]:
df_taxonomy[(df_taxonomy['type'] != 'project') & (df_taxonomy['course_id'] == 4)].to_csv(os.path.join(cwd, '../df_taxonomy_falcon.csv'), sep=',')